In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv1D, BatchNormalization, Activation, Add, MaxPooling1D, GlobalAveragePooling1D, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.optimizers import Adam

In [2]:
DATA_PATH = os.path.join('dataset')
actions = np.array(['zero', 'one', 'two','three', 'four', 'five', 'six', 'seven', 'eight', 'nine'])
sequence_length = 100
num_features = 126
label_map = {label:num for num, label in enumerate(actions)}

In [3]:
sequences, labels = [], []

for action in actions:
    action_path = os.path.join(DATA_PATH, action)
    for sequence in np.array(os.listdir(action_path)).astype(int):
        frame_paths = [
            os.path.join(action_path, str(sequence), "{}.npy".format(frame_num))
            for frame_num in range(sequence_length)
        ]
        window = [np.load(frame_path) for frame_path in frame_paths]
        sequences.append(window)
        labels.append(label_map[action])

In [4]:
x = np.array(sequences)
y = to_categorical(labels).astype(int)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30)

In [5]:
log_dir = os.path.join('ResNet_Logs')
callback = TensorBoard(log_dir=log_dir)
ACCURACY_THRESHOLD = 0.95

class MyCallback(tf.keras.callbacks.Callback): 
    def __init__(self, monitor_metric='accuracy'):
        super(MyCallback, self).__init__()
        self.monitor_metric = monitor_metric
        self.threshold = ACCURACY_THRESHOLD

    def on_epoch_end(self, epoch, logs={}): 
        current_metric_value = logs.get(self.monitor_metric)
        if current_metric_value is not None and current_metric_value > self.threshold:
            print(f"\nReached {self.threshold * 100:.2f}% {self.monitor_metric}, stopping training!")
            self.model.stop_training = True

tb_callback = TensorBoard(log_dir=log_dir)
my_callback = MyCallback(monitor_metric='accuracy')

In [6]:
def residual_block(x, filters, kernel_size=3, stride=1):
    shortcut = x
    
    x = Conv1D(filters, kernel_size, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv1D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    
    shortcut = Conv1D(filters, 1, strides=stride, padding='same')(shortcut)
    
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    
    return x


In [7]:
model = Sequential()

model.add(Conv1D(64, kernel_size=7, padding='same', input_shape=(sequence_length, num_features)))
model.add(BatchNormalization())
model.add(Activation('relu'))

input_tensor = model.output

x = residual_block(input_tensor, filters=64)
x = residual_block(x, filters=64)
x = MaxPooling1D(2)(x)

x = residual_block(x, filters=128, stride=2)
x = residual_block(x, filters=128)
x = MaxPooling1D(2)(x)

x = residual_block(x, filters=256, stride=2)
x = residual_block(x, filters=256)
x = MaxPooling1D(2)(x)

x = residual_block(x, filters=512, stride=2)
x = residual_block(x, filters=512)
x = GlobalAveragePooling1D()(x)

x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
output_tensor = Dense(actions.shape[0], activation='softmax')(x)

model = Model(inputs=model.input, outputs=output_tensor)

In [8]:
optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name="Adam")
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [11]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=8, callbacks=[tb_callback, my_callback, early_stopping])

Epoch 1/100
175/175 [==============================] - 16s 28ms/step - loss: 2.6354 - accuracy: 0.1314 - val_loss: 2.4366 - val_accuracy: 0.0983
Epoch 2/100
175/175 [==============================] - 4s 24ms/step - loss: 2.3432 - accuracy: 0.1571 - val_loss: 2.2781 - val_accuracy: 0.2017
Epoch 3/100
175/175 [==============================] - 4s 23ms/step - loss: 2.1547 - accuracy: 0.2321 - val_loss: 2.2380 - val_accuracy: 0.2533
Epoch 4/100
175/175 [==============================] - 4s 23ms/step - loss: 2.0397 - accuracy: 0.2579 - val_loss: 1.9167 - val_accuracy: 0.3117
Epoch 5/100
175/175 [==============================] - 4s 23ms/step - loss: 1.9273 - accuracy: 0.3007 - val_loss: 2.0393 - val_accuracy: 0.2650
Epoch 6/100
175/175 [==============================] - 4s 24ms/step - loss: 1.7732 - accuracy: 0.3193 - val_loss: 2.4690 - val_accuracy: 0.2650
Epoch 7/100
175/175 [==============================] - 4s 24ms/step - loss: 1.6932 - accuracy: 0.3814 - val_loss: 1.5003 - val_accuracy

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 conv1d_input (InputLayer)      [(None, 100, 126)]   0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 100, 64)      56512       ['conv1d_input[0][0]']           
                                                                                                  
 batch_normalization (BatchNorm  (None, 100, 64)     256         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 100, 64)      0           ['batch_normalization[0][0]']

In [15]:
model.save('ResNet.h5')

In [41]:
model.load_weights('ResNet.h5')

In [13]:
yhat = model.predict(x_train)
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

print("Train Accuracy :-> ")
print(accuracy_score(ytrue, yhat)*100)

44/44 [==============================] - 1s 8ms/step
Train Accuracy :-> 
87.42857142857143


In [14]:
yhat = model.predict(x_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

print("Test Accuracy :-> ")
print(accuracy_score(ytrue, yhat)*100)

19/19 [==============================] - 0s 18ms/step
Test Accuracy :-> 
77.5
